# **Cricket Shot Action Model Creation**

Importing necessary packages

In [8]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
import numpy as np
import os
import cv2 as cv
import tensorflow as tf
import mediapipe as mp
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, BatchNormalization
from tensorflow.keras.callbacks import TensorBoard
from datetime import datetime
from sklearn.metrics import f1_score

Directory variables

In [3]:
# Basic Data properties
DATA_PATH = os.path.join('cricket_data')
actions = np.array(['drive', 'pull shot', 'ready', 'not ready', 'cut shot'])
no_sequences = 30
sequence_length = 30

Get Data Back

In [4]:
# Creating encoding
label_map = {label: num for num, label in enumerate(actions)}

sequences, labels = [], []

for action in actions:
    for sequence in range(no_sequences):
        window = []
        for frame_num in range(sequence_length):
            res = np.load(os.path.join(DATA_PATH, action, str(sequence), "{}.npy".format(frame_num)))
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])

X = np.array(sequences)  # shape = 150, 30, 132
y = to_categorical(labels).astype(int)

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)

In [14]:
# Actual Neural Network
model = Sequential()
model.add(LSTM(256, return_sequences=True, activation='tanh', input_shape=(30, 132)))
model.add(Dropout(0.3))
model.add(BatchNormalization())
model.add(LSTM(128, return_sequences=True, activation='tanh'))
model.add(Dropout(0.3))
model.add(BatchNormalization())
model.add(LSTM(64, return_sequences=False, activation='tanh'))
model.add(Dropout(0.25))
model.add(BatchNormalization())
model.add(Dense(32, activation='tanh'))
model.add(Dropout(0.25))
model.add(Dense(16, activation='tanh'))
model.add(Dropout(0.2))
model.add(Dense(actions.shape[0], activation='softmax'))

model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=110, callbacks=[tb_callback], batch_size=16)
date_time = datetime.now().strftime("%Y%m%d_%H%M%S")
model.save(f'model{date_time}.h5')
print("Saved the model")

loss, accuracy = model.evaluate(X_test, y_test)
print(accuracy)

Epoch 1/110


c:\Users\lordw\OneDrive\Documents\Python_Master\Coding_Projects\cricket_shots\my_env\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


7/7 ━━━━━━━━━━━━━━━━━━━━ 5s 39ms/step - accuracy: 0.3221 - loss: 1.6903
Epoch 2/110
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step - accuracy: 0.5606 - loss: 1.1132
Epoch 3/110
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step - accuracy: 0.7520 - loss: 0.8494
Epoch 4/110
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step - accuracy: 0.7655 - loss: 0.7960
Epoch 5/110
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step - accuracy: 0.8481 - loss: 0.6611
Epoch 6/110
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step - accuracy: 0.9486 - loss: 0.5129
Epoch 7/110
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step - accuracy: 0.9365 - loss: 0.4678
Epoch 8/110
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step - accuracy: 0.9154 - loss: 0.5427
Epoch 9/110
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step - accuracy: 0.9373 - loss: 0.4847
Epoch 10/110
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step - accuracy: 0.9010 - loss: 0.4597
Epoch 11/110
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step - accuracy: 0.9186 - loss: 0.5203
Epoch 12/110
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step - accuracy: 0.9700 - loss: 0.2962
E

Saved the model
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - accuracy: 0.9649 - loss: 0.1571 
0.9473684430122375


In [15]:
# 1. Predict the probabilities for the test set
y_pred_prob = model.predict(X_test)

# 2. Convert predicted probabilities to class labels
y_pred = np.argmax(y_pred_prob, axis=1)

# 3. Convert one-hot encoded y_test to class labels (if applicable)
y_test_class = np.argmax(y_test, axis=1)

# 4. Calculate the F1 score
f1 = f1_score(y_test_class, y_pred, average='weighted')

print("F1 Score:", f1)

2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 503ms/step
F1 Score: 0.9471837488457988
